In [ ]:
from datetime import datetime
from dateutil import relativedelta

now = datetime(year=2023, month=12, day=18, hour=12, minute=18, second=0)

print('현재시간:' + str(now))
print('------------월 연산-------------')
print(now + relativedelta.relativedelta(month=1))
print(now.replace(month=1))

In [ ]:
import snowflake.connector
import pandas as pd

# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

try:
    # 커서 생성
    cursor = conn.cursor()

    # SQL 쿼리 실행
    query = "SELECT COUNT(*) FROM DW_LOAD_DB.HDHS_CU.CU_ITNT_CUST_GRD_1_INF"  # 조회할 테이블과 조건
    query2 = "alter stage DW_LOAD_DB.CONFIG.STG_AWS_S3 refresh"  # 조회할 테이블과 조건
    query3 = f"""
        SELECT 
           *
        FROM 
            DW_LOAD_DB.CONFIG.V_S3_LIST A 
        """
    cursor.execute(query)
    print (cursor.fetchall())
    
    cursor.execute(query2)
    print (type(cursor.fetchone()[1]))
    
    colums = [row[0] for row in cursor.fetchall()]
    df = pd.read_sql(query3, conn)

    # 결과 출력

finally:
    # 연결 닫기
    cursor.close()
    conn.close()
    
df

In [ ]:
df =pd.DataFrame(df)

In [ ]:
df

In [ ]:
df.to_parquet('df.parquet',engine='pyarrow', index=False)

In [ ]:
df['TABLE_NAME'][0]

In [ ]:
# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

cursor = conn.cursor()

# for i,row in df.iterrows():
#     if df['TABLE_SCHEMA'][i] == "HDHS_OD" and df['TABLE_NAME'][i] != "OD_BASKT_INF":
#         
#         query5 = f"""
#         CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB('DW_LOAD_DB','HDHS_OD','{df['TABLE_NAME'][i]}','{df['FILE_NAME']}','20241223120000','20241223150000')
#         """
#         print(query5)
#         cursor.execute(query5)
#         print(cursor.fetchall())
        
query5 = f""" 
    CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
        'DW_LOAD_DB',
        'HDHS_OD',
        '{df['TABLE_NAME'][1]}',""" + str("''2024/12/23/CU_ARS_LDIN_MST_CRYPT_20241223_030000.csv','2024/12/23/20241223_040000.csv''") +""",
        '20241223120000',
        '20241223150000'
    )
    """
print(query5)
cursor.execute(query5)
print(cursor.fetchall())
        
        
cursor.close()
conn.close() 

In [ ]:
fmrom airflow.providers.snowflake.operators.snowflake import SnowflakeOperator

CALL_PROCEDURE_QUERY = """
CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
    'DW_LOAD_DB',
    'HDHS_OD',
    '{{ params.table_name }}',
    '{{ params.file_name }}',
    '{{ params.start_time }}',
    '{{ params.end_time }}'
)
"""

call_procedure = SnowflakeOperator(
    task_id='call_snowflake_procedure',
    snowflake_conn_id='my_snowflake_conn',
    sql=CALL_PROCEDURE_QUERY,
    params={
        'table_name': {df['TABLE_NAME'][1]},
        'file_name': {df['FILE_NAME'][1]},
        'start_time': '20241223120000',
        'end_time': '20241223150000'
    },
)

print(call_procedure)


In [1]:
import pandas as pd

src_columns =['PROC_TIME','TRED','CALL_ID','MESG','Q_WAIT_TIME','STON','VDN_DIAL_NO','UCID','IN_CALL_NO']
tgt_columns =['UCID','ACD','PROC_DATE','PROC_TIME','TRED','CALL_ID','MESG','Q_WAIT_TIME','STON','VDN_DIAL_DVCD','VDN_DIAL_NO','IN_CALL_NO','ETL_DTM']
file_path="hmall_1225.dmp"
# 파일을 Pandas DataFrame으로 변환
try:
    # 쉼표로 구분된 데이터를 읽기 (UTF-8 또는 EUC-KR 등의 인코딩을 시도)
    df = pd.read_csv(file_path, encoding='utf-8', header=None)
except UnicodeDecodeError:
    # 만약 UTF-8로 읽히지 않으면 EUC-KR로 재시도
    df = pd.read_csv(file_path, encoding='euc-kr', header=None)

df



,0,1,2,3,4,5,6,7,8,9,10,11
0,00:00:00.146,2140,c6100,상담연결(AgentVDN->Agent),BSR,i01093101121,v60324,U10001044731703429962,A01093101121,IN[],CO[],END
1,00:00:01.146,2328,c6097,상담연결(AgentVDN->Agent),,i01084865650,v60717,U10002060531703429968,A01084865650,IN[30000],CO[],END
2,00:00:02.114,22a8,c6090,상담연결(AgentVDN->Agent),,i01029385307,v60105,U10002060901703429993,A01029385307,IN[],CO[],END
3,00:00:02.614,17e4,c6010,호종료,,s3167,v,U,A,IN[],CO[],END
4,00:00:02.614,205c,c5801,수신(Inbound),,s3167,v3416,U10002058011703429734,A01073785912,IN[000],CO[],END
...,...,...,...,...,...,...,...,...,...,...,...,...
118551,23:45:58.331,0dc8,c2656,상담연결(AgentVDN->Agent),,i10512,v62044,U10002026541735137940,A0618633452,IN[20100],CO[],END
118552,23:58:01.657,2fc0,c,IVR상담연결(IVR->CTI),,i10085,v64601,U,A01021140996,IN[100V0],CO[],END
118553,23:58:44.695,30ec,c,IVR상담연결(IVR->CTI),,i10184,v62036,U,A01075774322,IN[100V0],CO[],END
118554,23:58:46.398,3370,c2900,상담연결(AgentVDN->Agent),,i10184,v62036,U10002028831735138697,A01075774322,IN[100V0],CO[],END
